____
1) Написать функцию с декоратором которая логирует аргументы и возвращает нам значение функции по умножению x на y.     
Write a Python program to create a decorator that logs the arguments and return value of a function.

In [ ]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print(f'{func.__name__} с аргументами {args} и {kwargs}')
        result = func(*args, **kwargs)
        print(f'{func.__name__} возвращает результат {result}')
        return result
    return wrapper

@decorator
def multiply(a, b):
    return a * b

multiply(10, 2)

____
2) Написать декоратор, который считает время выполнения функции перемножения всех цифр из списка.   
Write a Python program to create a decorator function to measure the execution time of a function.

In [ ]:
import time

def decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f'Функция {func.__name__} заняла {execution_time:.4f} секунд')
        return result
    return wrapper

@decorator
def multiply(nums):
    result = 1
    for num in nums:
        result *= num
    return result

multiply([1,2,3,4,5,6,7,8])

____
3) Написать функцию, которая декорирует умножение x на y, но при этом кэширует значение. И если значение уже было создано, выдает нам из кэша.   
Write a Python program that implements a decorator to cache the result of a function.   

In [ ]:
def decorator(func):
    cache = {}
    def wrapper(*args, **kwargs):
        key = (*args, *kwargs.items())
        if key in cache:
            print(f'Возвращаем посчитанное значение из кэша:')
            return cache[key]
        result = func(*args, **kwargs)
        cache[key] = result

        return result
    return wrapper


@decorator
def multiply(x,y):
    print('Умножаем x на y...')
    return x * y

print(multiply(2,5))
print(multiply(2,5))
print(multiply(4,5))

____
4) Декоратор, который считает оборачивает функцию куба от числа, но если число больше нуля. Если меньше - вывести ошибку.   
Write a Python program that implements a decorator to validate function arguments based on a given condition.

In [ ]:
def positive_only(func):
    def wrapper(x):
        if x <= 0:
            raise ValueError("Число x должно быть положительным")
        return func(x)
    return wrapper

@positive_only
def cube(x):
    return x ** 3


# Или такое, более универсальное решение
def validate(condition):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if condition(*args, **kwargs):
                return func(*args, **kwargs)
            else:
                raise ValueError('Число х должно быть больше 0')
        return wrapper
    return decorator


@validate(lambda x: x > 0)
def cube(x):
    return x ** 3

cube(3)

____
5) Rate-limiter, который ограничивает скорость вызовов функции. Мини-версию механизма, который используют API Instagram, Telegram, Discord, биржи и т.д. @rate_limits(max_calls=6, period=10)

In [ ]:
import time

def rate_limits(max_calls, period):
    def decorator(func):
        calls = 0
        last_reset = time.time()

        def wrapper(*args, **kwargs):
            nonlocal calls, last_reset
            elapsed = time.time() - last_reset
            if elapsed > period:
                calls = 0
                last_reset = time.time()
            if calls >= max_calls:
                raise Exception("Rate limit exceeded. Please try again later.")
            calls += 1
            return func(*args, **kwargs)

        return wrapper
    return decorator

@rate_limits(max_calls=6, period=10)
def api_call():
    print("API call executed successfully...")

for _ in range(8):
    try:
        api_call()
    except Exception as e:
        print(f"Error occurred: {e}")

____
6) Попытка подключиться к бд sqlite. Сделать декоратор, который будет переподключаться при неудачных попытках подключиться к БД.   
Write a Python program that implements a decorator to retry a function multiple times in case of failure.

In [ ]:
import sqlite3
import time

def retry_on_failure(max_retries, delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(max_retries):
                try:
                    result = func(*args, **kwargs)
                    return result
                except Exception as e:
                    print(f"Error occurred: {e}. Retrying...")
                    time.sleep(delay)
            raise Exception("Maximum retries exceeded. Function failed.")

        return wrapper
    return decorator

@retry_on_failure(max_retries=3, delay=2)
def connect_to_database():
    conn = sqlite3.connect("example.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    return result

try:
    data = connect_to_database()
    print("Data retrieved successfully:", data)
except Exception as e:
    print(f"Failed to connect to the database: {e}")

____
7) Написать прогу которая декорирует функцию, которая делит два числа. Но чтобы у нее был базовый ответ на любую ошибку деления, а декоратор чтобы выдавал какая именно ошибка благодаря конструкии `try except`

In [ ]:
def handle_exception(default_response):
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                return func(*args,**kwargs)
            except Exception as e:
                print(f'Exception occured: {e}')
                return default_response
        return wrapper
    return decorator


@handle_exception(default_response = 'An error occured')
def devide_nums(x, y):
    return int(x / y)

result = devide_nums(7, 0)
print(result)